In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
#Change the following line to run this code on your own data.

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [30]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    x, states = self.gru(x, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x



In [31]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [33]:
model.summary()

Model: "my_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (64, 100, 256)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (64, 100, 66)               │          67,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [35]:
sampled_indices

array([22, 54, 23, 50, 36, 43, 33,  2,  4, 54, 24, 55,  5, 47, 40, 32, 58,
       29,  4, 62, 47, 38, 20, 56, 38,  8, 27, 56, 19, 13, 41, 15, 27, 53,
       38, 38, 63, 53, 51, 51, 24,  1, 60, 35, 45, 32, 48, 17, 40, 64,  4,
       49, 62, 40,  8,  7, 43, 53, 55,  5, 39,  3,  4, 62, 28, 18,  1, 35,
       15, 35,  4, 53, 36, 51, 38, 11,  7, 62, 35, 27,  5, 27,  9, 53, 33,
        8, 35,  6, 37, 57, 40,  9, 50, 28, 37, 35,  5, 53, 17, 56],
      dtype=int64)

In [36]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'onfidence\nThan boys pursuing summer butterflies,\nOr butchers killing flies.\n\nMENENIUS:\nYou have made'

Next Char Predictions:
 b"IoJkWdT $oKp&haSsP$whYGqY-NqF?bBNnYYxnllK\nuVfSiDay$jwa-,dnp&Z!$wOE\nVBV$nWlY:,wVN&N.nT-V'Xra.kOXV&nDq"


In [37]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [38]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1912336, shape=(), dtype=float32)


In [39]:
tf.exp(example_batch_mean_loss).numpy()

66.10429

In [40]:
# Next time, try it with 'metrics=["accuracy"]'

model.compile(optimizer='adam', loss=loss, metrics=["accuracy"])

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Adding `.weights.h5` suffix
suffix = '.weights.h5'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}", suffix)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [44]:
EPOCHS = 20

In [45]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 75s 424ms/step - loss: 3.0397
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 75s 430ms/step - loss: 1.9095
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 75s 431ms/step - loss: 1.6270
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 74s 425ms/step - loss: 1.4812
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 73s 421ms/step - loss: 1.3939
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 73s 421ms/step - loss: 1.3308
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 73s 423ms/step - loss: 1.2823
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 74s 428ms/step - loss: 1.2344
Epoch 9/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 74s 427ms/step - loss: 1.1932
Epoch 10/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 74s 427ms/step - loss: 1.1547
Epoch 11/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 75s 434ms/step - loss: 1.1117
Epoch 12/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 75s 434ms/step - loss: 1.0721
Epoch 13/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 74s 430ms/step - loss: 1.0257
Epoch 14/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 76s 437ms/step - loss: 0.9787
Epoch 15/20
172

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [48]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
ARirin?
Ty fo tr

ASAn the bese mal hy tin o gr kio wa Whearan ta and tld wamive?

KELolt esco pl wo tininor SCAdonth IELI gime d.


TRO:

Towhay be G shard t d tcras iowardg ainyodstitousin AS:
Whait nkir ag

He; wo that.
T:

ELit th.

TERY:

WIOLAn an bea f pelousofore t demer waconar:
FRifrp mo,
CELon n amiour t be Yo, prak aMot mashed bifiby's igege.
Hel thais thilerover aree bese
Hove an ongrd INO matofule id, gn? toway,

Tha hars! bo nerosen thoushe ldraper y d hee thes pe! kst D bithithatred as, th SUn o.
IONG d'stind there ars 's, m, soutivethoun ld gangnoureeaine'd, they,
An'l t wirje w ar t t Yofre g,
Yof shils t it?
By apr.
II mest, ir inshengouns, maldere s,
Y.
Co vesh, h d ar me?
AME t ifow. hee cucopeve bse;

Hevessoch me ours pet ber,
Houn iong chy, sere, clancagees ow fishano chines taingiere! mothargo d bore t to on, fe med yopanghourendes weatar amyove tha pl moud minouresend Prds F


FLO:

Pouse thtt thit cet pe t tods bre thofinavicus sunerutr de edst.

HA po

In [49]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Sen ur?
Windyougnckisco sw wis an. iman t,
Sed ta man ar dosonem ge yont
Theveld t Itise

Ar me thye me eno cit hyomend toven ace he t thar s mo m A:
pll milll equ pind ave t mes ce tise pize now mous thild animandatis inoneno thman,
I:

Th w serdore ath aMarinwir ve,
Cof mowe

I h s handertcl th,
I t is, y thyind CING y cu, gerus ore tit apop cheshery,


Tusward;
Chtsesimeche h, owifothithocamamand MI
Deered m's toucheakir,

Anof ul-brncqur' seabreyoose my.
TEd morethe?
NINTESTEYondousche the leangred
MI on fo t med wan bamotat ke:
Se a an fay toucer ndochanote S:
Theverdard m-t k! my; l p, ILer, is,
BRO:
T; plal ld brthoth m, y thacet K:
Whyonst mbusizer, ain
Gor?


Thil ny G-bocorees ineathet miler, le por thon me ceve y tharane wilar t, IRANELAno sthod me show thack er thatheey thy mees o kick d! me tofur bout tiseve knothad whigerkes.
OURINIss y prpoond?
by, my my mand ta thororanced.
FRIO:
Fu you ared ratwos,


TI ck th tho, y CUCod phey t gre, wndery, d hor-ge' anowal th 